In [7]:
!pip install google-cloud-storage


In [5]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from zipfile import ZipFile
from io import BytesIO
from google.cloud import storage
from io import StringIO
import requests
import zipfile
import os

In [3]:


# URL of the ZIP file
url = "https://bikeshare.metro.net/wp-content/uploads/2024/04/metro-trips-2024-q1.zip"

# Specify user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'

# Make request with specified user-agent
response = requests.get(url, headers={'User-Agent': user_agent})

# Check if request was successful
if response.status_code == 200:
    # Read the content of the ZIP file
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        # Assume there's only one CSV file in the ZIP archive
        csv_filename = z.namelist()[0]
        with z.open(csv_filename) as f:
            # Read CSV file into a DataFrame
            df_raw = pd.read_csv(f)

    # Display info and shape of the DataFrame
    print(df_raw.info())
    print(df_raw.shape)
    # Display the first few rows of the DataFrame
    print(df_raw.head())
else:
    print(f"Failed to download ZIP file from URL: {url}. Response status code: {response.status_code}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119849 entries, 0 to 119848
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   trip_id              119849 non-null  int64  
 1   duration             119849 non-null  int64  
 2   start_time           119849 non-null  object 
 3   end_time             119849 non-null  object 
 4   start_station        119849 non-null  int64  
 5   start_lat            119840 non-null  float64
 6   start_lon            119840 non-null  float64
 7   end_station          119849 non-null  int64  
 8   end_lat              117663 non-null  float64
 9   end_lon              117663 non-null  float64
 10  bike_id              119849 non-null  int64  
 11  plan_duration        119849 non-null  int64  
 12  trip_route_category  119849 non-null  object 
 13  passholder_type      119849 non-null  object 
 14  bike_type            119849 non-null  object 
dtypes: float64(4), in

In [17]:
from io import StringIO
import os
from google.cloud import storage

def upload_blob(bucket_name, data, destination_blob_name):
    """Uploads data to the bucket as a blob."""
    # Initialize a client with explicit credentials
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"

    # Create a storage client object
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # Create a blob object
    blob = bucket.blob(destination_blob_name)

    # Upload the data
    blob.upload_from_string(data)

    print(f'Data uploaded to {destination_blob_name} in bucket {bucket_name}.')

# Set your Google Cloud Storage bucket name
bucket_name = "bikeshare_project_data"

# Set the destination blob name (file name in Google Cloud Storage)
destination_blob_name = "bikeshare.csv"

# Convert DataFrame to CSV format in memory
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Call the function to upload the data
upload_blob(bucket_name, data, destination_blob_name)


Data uploaded to bikeshare.csv in bucket bikeshare_project_data.


In [13]:
import pandas as pd
import requests
from io import BytesIO

# Specify the URL
url = "https://bikeshare.metro.net/wp-content/uploads/2024/04/metro-bike-share-stations-2024-04-01.csv"

# Specify user-agent
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'

# Make request with specified user-agent
response = requests.get(url, headers={'User-Agent': user_agent})

# Check if request was successful
if response.status_code == 200:
    # Read CSV file directly from the URL with specified encoding
    station_raw = pd.read_csv(BytesIO(response.content), encoding='latin1')

    # Display info and shape of the DataFrame
    print(station_raw.info())
    print(station_raw.shape)
    # Display the first few rows of the DataFrame
    print(station_raw.head())
else:
    print(f"Failed to download CSV file from URL: {url}. Response status code: {response.status_code}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419 entries, 0 to 418
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Station_ID           413 non-null    float64
 1   Station_Name         413 non-null    object 
 2   Day of Go_live_date  413 non-null    object 
 3   Region               412 non-null    object 
 4   Status               413 non-null    object 
 5   Latitude             413 non-null    float64
 6   Longitude            413 non-null    float64
 7   Unnamed: 7           0 non-null      float64
 8   Unnamed: 8           0 non-null      float64
 9   Unnamed: 9           0 non-null      float64
 10  Unnamed: 10          0 non-null      float64
 11  Unnamed: 11          0 non-null      float64
 12  Unnamed: 12          0 non-null      float64
 13  Unnamed: 13          0 non-null      float64
 14  Unnamed: 14          0 non-null      float64
 15  Unnamed: 15          0 non-null      flo

In [16]:
from io import StringIO
import os
from google.cloud import storage

def upload_blob(bucket_name, data, destination_blob_name):
    """Uploads data to the bucket as a blob."""
    # Initialize a client with explicit credentials
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"

    # Create a storage client object
    storage_client = storage.Client()

    # Get the bucket
    bucket = storage_client.bucket(bucket_name)

    # Create a blob object
    blob = bucket.blob(destination_blob_name)

    # Upload the data
    blob.upload_from_string(data)

    print(f'Data uploaded to {destination_blob_name} in bucket {bucket_name}.')

# Set your Google Cloud Storage bucket name
bucket_name = "bikeshare_project_data"

# Set the destination blob name (file name in Google Cloud Storage)
destination_blob_name = "station.csv"

# Convert DataFrame to CSV format in memory
output = StringIO()
station_raw.to_csv(output, index=False)
station_data = output.getvalue()
output.close()

# Call the function to upload the data
upload_blob(bucket_name, station_data, destination_blob_name)


Data uploaded to station.csv in bucket bikeshare_project_data.
